In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
data_url = '.../Deloitte Football Money League - Data.csv'
bg_img_url = '.../grass.jpg'
icons_url = '.../Icons/'

# **Import and Prep**

In [0]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import matplotlib.animation as animation
import matplotlib.patheffects as path_effects
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.collections import PatchCollection

from PIL import Image, ImageOps, ImageDraw
from IPython.display import HTML

import datetime as dt
import warnings
warnings.filterwarnings("ignore")

## General functions

In [0]:
def create_fig(figsize, left_offset=0.2, heights=(0.15, 0.8, 0.05)):
  """
  Figure template for production charts, for 1 subplot not inclusive of header and footer
  """
  fig = plt.figure(figsize=figsize)
  widths = [left_offset, 1-left_offset]
  gs = fig.add_gridspec(ncols=2, nrows=3, width_ratios=widths, height_ratios=heights)
  ax_title = fig.add_subplot(gs[0, :])
  ax_footer = fig.add_subplot(gs[2, :])
  axes = []
  for i in range(2):
    axes.append(fig.add_subplot(gs[1, i]))
  for ax in [ax_title, ax_footer, axes[0]]:
    ax.axis('off')

  return fig, axes, ax_title, ax_footer

def set_background_image(bg_img_loc, offset=0, alpha=0.2):
  """
  Sets image as fig background
  """
  bg_ax = fig.add_axes([-offset, -offset, 1+2*offset, 1+2*offset])
  bg_ax.set_zorder(0.1)
  bg_ax.set_xlim([0,1])
  bg_ax.set_ylim([0,1])
  bg_ax.axis('off')
  bg_ax.imshow(np.flipud(plt.imread(bg_img_loc)[:,:,:]), aspect='auto', extent=[0,1,0,1], origin='bottom', alpha=alpha)

def virtual_rotate(x, y, angle=0):
  """
  Simple function to virtually rotate a line by [angle] degrees clockwise. Pass in coordinates of starting line in form [x_0, x_1], [y_0, y_1], 
  function returns coordinates of rotated end point [x_new, y_new]. Initial line must be vertical
  """
  x_0, x_1 = x[0], x[1]
  y_0, y_1 = y[0], y[1]

  if x_0 != x_1:
    raise ValueError("This is only a simple function")
  
  length = y_1 - y_0
  x_new = x_0 + np.sin(np.deg2rad(angle))*length
  y_new = y_0 + np.cos(np.deg2rad(angle))*length
  
  return [x_0, x_new], [y_0, y_new]

## Specific helper functions

In [0]:
def create_dff(data, frame):
  """
  Returns dataset filtered for one point in time (e.g. one year or one day depending on unit of 'frame')
  """
  return data[data[frame_col]==frame].sort_values(by=value_col, ascending=False).head(max_bars).reset_index(drop=True) 

def custom_position_width(num_bars):
  """
  Customise positions and widths so that top 3 bars have extra weighting. If num_bars=20, positions 11-20 have reduced weighting
  """
  if num_bars <= 15:
    default_width = 100/num_bars
    new_w_top3 = default_width * 4/3
    new_w_rest = (100-4*default_width)/(num_bars-3)
    new_pos_top3 = np.linspace(100-0.5*new_w_top3, 100-2.5*new_w_top3, 3)
    new_pos_rest = np.linspace(100-4*default_width-0.5*new_w_rest, 0.5*new_w_rest, num_bars-3)
    
    new_positions = np.concatenate((new_pos_top3, new_pos_rest))
    new_widths = [new_w_top3]*3 + [new_w_rest]*(num_bars-3)
  
  else:
    if num_bars != 20:
      raise ValueError("Inappropriate number of bars")
    new_w_top3 = 22.5/3
    new_w_top10 = 37.5/7
    new_w_rest = 40/10
    new_pos_top3 = np.linspace(100-0.5*new_w_top3, 77.5+0.5*new_w_top3, 3)
    new_pos_top10 = np.linspace(77.5-0.5*new_w_top10, 40+0.5*new_w_top10, 7)
    new_pos_rest = np.linspace(40-0.5*new_w_rest, 0.5*new_w_rest, 10)
    
    new_positions = np.concatenate((new_pos_top3, new_pos_top10, new_pos_rest))
    new_widths = [new_w_top3]*3 + [new_w_top10]*7 + [new_w_rest]*10

  return new_positions, new_widths

def edit_cols(dff, custom=False):
  """
  Removes redundant columns from dff and adds new columns for positions, widths and frame strings
  """
  num_bars = len(dff)

  if custom == True:
    positions, widths = custom_position_width(num_bars)  
  else:
    width = 100/num_bars
    positions = np.linspace(100-width/2, width/2, num_bars)
    widths = [width]*num_bars
  
  dff['Position'] = positions
  dff['Width'] = widths
  dff['Width'] = 0.8 * dff['Width']
  dff['Frame'] = dff[frame_col].astype('str')

  return dff[[frame_col, label_col, value_col, color_key_col, 'Position', 'Width', 'Frame']]

def interpolate(data, frame, split=1, intervals=10, custom=False):
  """
  Takes data for one point in time (data[frame]) and interpolates values, positions and widths between this point and the next point in time (data[frame+1]), 
  by splitting the selected time period [split] times with interpolated values for each split checkpoint, then interpolating position and width for [intervals] intervals
  between each split. E.g. split = 3, intervals = 10 would slice the time period at 0.25, 0.5 and 0.75, creating 4 segments, then slices each segment into a further 10 intervals.
  For interpolated intervals, 'zorder' column added so that rising bars have higher zorder and dropping bars have lower zorder.
  Returns list of (split+1)*intervals dataframes containing interpolated values, positions and widths. 
  """
  next_frame = data[frame_col].unique().tolist()[ data[frame_col].unique().tolist().index(frame)+1 ]
  df_0 = edit_cols(create_dff(data, frame), custom=custom)[[frame_col, label_col, value_col, color_key_col, 'Position', 'Width']]
  df_1 = edit_cols(create_dff(data, next_frame), custom=custom)[[frame_col, label_col, value_col, color_key_col, 'Position', 'Width']]
  bars = list( set( pd.concat([df_0[label_col], df_1[label_col]]) ) )

  checkpoints = [df_0]
  for i in range(1, split+1):
    df_temp = pd.DataFrame().reindex_like(df_0)

    for index, bar in enumerate(bars):
      if index >= len(df_temp):
        df_temp.append(pd.Series(), ignore_index=True)

      if bar in df_0[label_col].values and bar in df_1[label_col].values:
        start_value = df_0[df_0[label_col]==bar][value_col].iloc[0]
        end_value = df_1[df_1[label_col]==bar][value_col].iloc[0]
      if bar in df_0[label_col].values and bar not in df_1[label_col].values:
        start_value = df_0[df_0[label_col]==bar][value_col].iloc[0]
        end_value = 0.95*df_1[value_col].min()
      if bar in df_1[label_col].values and bar not in df_0[label_col].values:
        start_value = 0.95*df_0[value_col].min() if frame != 2005 else 0.95*df_1[value_col].min()
        end_value = df_1[df_1[label_col]==bar][value_col].iloc[0]
        
      value_diff = end_value - start_value
      df_temp.loc[index, label_col] = bar
      df_temp.loc[index, value_col] = round(start_value + value_diff * i/(split+1), 1)
      df_temp.loc[index, color_key_col] = df_0[df_0[label_col]==bar][color_key_col].iloc[0] if bar in df_0[label_col].values else df_1[df_1[label_col]==bar][color_key_col].iloc[0]
        
    df_temp = df_temp.sort_values(by=value_col, ascending=False).reset_index(drop=True)
    df_temp[frame_col] = df_0[frame_col][0]
    df_temp['Position'] = df_1['Position']
    df_temp['Width'] = df_1['Width']
    df_temp.fillna(0, inplace=True)

    checkpoints.append(df_temp)
  
  interp_dfs = []
  for checkpoint_idx in range(len(checkpoints)):
    df_start = checkpoints[checkpoint_idx]
    df_end = checkpoints[checkpoint_idx+1] if checkpoint_idx+1 < len(checkpoints) else df_1

    for j in range(1, intervals):
      df_temp2 = pd.DataFrame().reindex_like(df_end)

      for index, bar in enumerate(bars):
        if bar in df_start[label_col].values and bar in df_end[label_col].values:
          start_value = df_start[df_start[label_col]==bar][value_col].iloc[0]
          end_value = df_end[df_end[label_col]==bar][value_col].iloc[0]
          start_pos = df_start[df_start[label_col]==bar]['Position'].iloc[0]
          end_pos = df_end[df_end[label_col]==bar]['Position'].iloc[0]
          start_width = df_start[df_start[label_col]==bar]['Width'].iloc[0]
          end_width = df_end[df_end[label_col]==bar]['Width'].iloc[0]
        if bar in df_start[label_col].values and bar not in df_end[label_col].values:
          start_value = df_start[df_start[label_col]==bar][value_col].iloc[0]
          end_value = 0.95*df_end[value_col].min()
          start_pos = df_start[df_start[label_col]==bar]['Position'].iloc[0]
          end_pos = 0
          start_width = df_start[df_start[label_col]==bar]['Width'].iloc[0]
          end_width = df_end['Width'].min()
        if bar in df_end[label_col].values and bar not in df_start[label_col].values:
          start_value = 0.95*df_start[value_col].min() if frame != 2005 else 0.95*df_end[value_col].min()
          end_value = df_end[df_end[label_col]==bar][value_col].iloc[0]
          start_pos = 0
          end_pos = df_end[df_end[label_col]==bar]['Position'].iloc[0]
          start_width = df_start['Width'].min()
          end_width = df_end[df_end[label_col]==bar]['Width'].iloc[0]
        
        value_diff = end_value - start_value
        pos_diff = end_pos - start_pos
        width_diff = end_width - start_width
        df_temp2.loc[index, label_col] = bar
        df_temp2.loc[index, value_col] = round(start_value + value_diff * j/intervals, 1)
        df_temp2.loc[index, color_key_col] = df_0[df_0[label_col]==bar][color_key_col].iloc[0] if bar in df_0[label_col].values else df_1[df_1[label_col]==bar][color_key_col].iloc[0]
        df_temp2.loc[index, 'Position'] = start_pos + pos_diff * j/intervals
        df_temp2.loc[index, 'Width'] = start_width + width_diff * j/intervals if df_temp2.loc[index, 'Position'] > 0 else 0
        df_temp2.loc[index, 'Zorder'] = 200 if pos_diff > 0 else 100
      
      df_temp2 = df_temp2.sort_values(by=value_col, ascending=False).reset_index(drop=True)
      df_temp2[frame_col] = df_0[frame_col][0]
      
      interp_dfs.append(df_temp2)

  if len(checkpoints)*(intervals-1) != len(interp_dfs):
    raise ValueError("Interpolation error")

  outputs = interp_dfs.copy()
  for index, pos in enumerate( np.linspace(len(interp_dfs) - len(interp_dfs)/len(checkpoints), 0, len(checkpoints)) ):
    outputs.insert(int(pos), checkpoints[-(index+1)])

  for index, df in enumerate(outputs):
    df['Frame'] = "{}_{}".format(df[frame_col][0], index)
    
  return outputs

def generate_frames(data, split=1, intervals=10):
  """
  Combine original data with interpolated data to output a list of frames to be animated. Make sure split and intervals is consistent with interpolate
  """
  original_frames = list(map(str, data[frame_col].unique()))
  output_frames = []

  for frame in original_frames[:-1]:
    output_frames.extend( [frame + "_0"]*5 )
    
    for i in range(1, (split+1)*intervals):
      output_frames.append( "{}_{}".format(frame, i) )
  
  output_frames.extend( [original_frames[-1]]*30 )

  if len(output_frames) != (len(original_frames) - 1)*((split+1)*intervals + 4) + 30:
    raise ValueError("Error generating frames")

  return output_frames

# **Project: Deloitte Football Money League**

In [0]:
df = pd.read_csv(data_url)
df.info()

In [0]:
### Input Data and Customise Parameters ###

data = df
frame_col = 'Year'
value_col = 'Revenue'
label_col = 'Club'
color_key_col = 'Country'
colors_dict = {'England':'#ffffff', 'Spain':'#e60026', 'Italy':'#009c3d', 'France':'#123163', 'Germany':'#161413', 
               'Scotland':'#c0c0c0', 'Russia':'#c0c0c0', 'Portugal':'#c0c0c0', 'Turkey':'#c0c0c0', 'Netherlands':'#c0c0c0', 'Brazil':'#c0c0c0'}
max_bars = 20
frame_split = 3
frame_dtype = 'int'

#-------------------------------------------------------------------------------#
### Preparation of Data ###

original_frames = data[frame_col].unique().tolist()
keys, dffs = [], []
for f in original_frames[:-1]:
  list_of_dffs = interpolate(data, f, split=frame_split, custom=True)
  dffs.extend(list_of_dffs)
  keys.extend([ dff['Frame'][0] for dff in list_of_dffs ])

dffs.append( edit_cols(create_dff(data, original_frames[-1]), custom=True) )
keys.append( str(original_frames[-1]) )

dffs_dict = dict(zip(keys, dffs))

if len(dffs_dict) != (len(original_frames) - 1)*(frame_split + 1)*10 + 1:
  raise ValueError("Error creating dffs")

In [0]:
#-------------------------------------------------------------------------------#
### Fixed Sections of Chart ###

fig, axes, ax_title, ax_footer = create_fig(figsize=(7.5,7.5), left_offset=0.18, heights=(0.1,0.9,0))
fig.set_dpi(96)
ax, ax2 = axes[1], axes[0]
for axis in [ax, ax2, ax_title, ax_footer]:
  axis.set_zorder(2)
  axis.patch.set_alpha(0)

set_background_image(bg_img_url, offset=0, alpha=0.4)

ax.set_ylim([0,100])
ax.yaxis.set_visible(False)
ax.xaxis.tick_top()
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('€%dm'))
ax.tick_params(axis='both', length=0, labelsize=7.5, labelcolor='darkslategray')
[spine.set_visible(False) for spine in ax.spines.values()]
ax2.set_ylim([0,100])
ax2.set_xlim([0,1])

font = 'Liberation Sans'
text_shadow = [path_effects.withSimplePatchShadow(offset=(0.8,-0.8), shadow_rgbFace='w', alpha=0.8)]
ax_title.text(0.01, 0.6, "Evolution of the Deloitte Football Money League", weight='bold', fontname=font, fontsize=16, path_effects=text_shadow)
ax_title.text(0.01, 0.2, "Measuring revenue generated from football operations by each club for each season from 2001-2019", fontname=font, fontsize=9.4, path_effects=text_shadow)
ax_footer.text(0, 0, "Data from DFML annual report", fontsize=7.5, path_effects=text_shadow)
ax_footer.text(1, 0, "Chart by @rrding1995, APR 2020", fontsize=7.5, ha='right', path_effects=text_shadow)

frame_label = ax.text(0.96, 0.15, "", fontsize=25, weight='bold', ha='right', va='bottom', path_effects=text_shadow, transform=ax.transAxes)
total_word = ax.text(0.96, 0.105, "Total:", fontsize=10, weight='bold', ha='right', va='center', path_effects=text_shadow, transform=ax.transAxes)
total_label = ax.text(0.96, 0.07, "", fontsize=13, color='darkorange', weight='bold', ha='right', va='center', path_effects=text_shadow, transform=ax.transAxes)
total_text = ax.text(0.96, 0.04, "", fontsize=6.5, weight='bold', ha='right', va='center', path_effects=text_shadow, transform=ax.transAxes)

for x in range(100, 1000, 100):
  ax.axvline(x, lw=0.8, color='darkslategray', alpha=0.4, zorder=0.9)

circle = patches.Circle((0.91, 0.28), radius=0.04, color='#252321', lw=3, fill=False, zorder=10, transform=ax.transAxes)
shadow = patches.Circle((0.911, 0.279), radius=0.04, color='w', lw=3, fill=False, zorder=9, transform=ax.transAxes)
[ax.add_patch(patch) for patch in [circle, shadow]]
x_start = [0.91, 0.91]
y_start = [0.28, 0.31]
clock_hand, = ax.plot(x_start, y_start, lw=3, color='#252321', solid_capstyle='round', zorder=10, transform=ax.transAxes)

bars, labels, values, logos = [], [], [], []

#-------------------------------------------------------------------------------#
### Functions for FuncAnimation ###

def draw_barchart(frame):
  """
  Input function for FuncAnimation to loop over
  """
  dff = dffs_dict[frame]

  if len(bars) != 0:
    for artist in (*bars, *labels, *values, *logos):
      artist.remove()

  xlim = dff[value_col].max()*1.14
  ax.set_xlim([0, xlim])

  rows = int( np.floor(len(dff)/10)*10 )
  frame_label.set_text("{}-{}".format( dff[frame_col][0]-2, str(dff[frame_col][0]-1)[-2:] ))
  total_label.set_text("€{:.2f}bn".format( dff.head(rows)[value_col].sum()/1000 ))
  total_text.set_text("generated by top {} clubs".format(rows))

  angle = 360 * int(frame.split("_")[1])/(10*(frame_split+1)) if frame != '2020' else 0
  x_rot, y_rot = virtual_rotate(x_start, y_start, angle=angle)
  clock_hand.set_data(x_rot, y_rot)
  
  bars.clear()
  labels.clear()
  values.clear()
  logos.clear()

  for index, row in dff.iterrows():
    if index < 3:
      label_color = 'darkslategray'
      text_weight = 'bold'
      text_size = 9.5
      logo_size = (30,30) if len(dff) > 19 else (40,40)
      logo_offset = 0.03 if len(dff) > 19 else 0.045
    if index >= 3 and index < 10:
      label_color = 'k'
      text_weight = 'normal'
      text_size = 8.5
      logo_size = (23,23) if len(dff) > 19 else (30,30)
      logo_offset = 0.025 if len(dff) > 19 else 0.035
    if index >= 10:
      label_color = '#545454'
      text_weight = 'normal'
      text_size = 7.5
      logo_size = (16,16) 
      logo_offset = 0.02
    try:
      zorder = row['Zorder']
    except:
      zorder = 100

    bar_color = colors_dict[row[color_key_col]]
    bar = ax.barh(row['Position'], row[value_col], height=row['Width'], color=bar_color, edgecolor=label_color, zorder=zorder, 
                  path_effects=[path_effects.withSimplePatchShadow(offset=(0.9,-0.9), shadow_rgbFace=label_color, alpha=0.75)])
    bars.append(bar)

    if row['Position'] != 0:
      label_text = ax2.text(1.05, row['Position'], row[label_col], fontname=font, ha='right', va='center', path_effects=text_shadow,
                            color=label_color, weight=text_weight, fontsize=text_size)
      value_text = ax.text(row[value_col], row['Position'], " €{}m".format(row[value_col]), fontname=font, va='center', path_effects=text_shadow,
                           color=label_color, weight=text_weight, fontsize=text_size)
    
      labels.append(label_text)
      values.append(value_text)

    if row['Width'] > 2:
      img = Image.open(icons_url+row[label_col]+'.png')
      img.thumbnail(logo_size)
      ab = AnnotationBbox(OffsetImage(img, zoom=72/fig.dpi), (row[value_col]-xlim*logo_offset, row['Position']), frameon=False)
      ab.set_zorder(zorder + 10)
      ax.add_artist(ab)
      logos.append(ab)

plt.tight_layout()
#-------------------------------------------------------------------------------#
### STATIC ###

#draw_barchart('2002_18')

#-------------------------------------------------------------------------------#
### ANIMATION ###

#animator = animation.FuncAnimation(fig, draw_barchart, frames=generate_frames(data, split=frame_split), interval=40)

#HTML(animator.to_jshtml()) 
#animator.save('test.mp4', dpi=96, fps=30, bitrate=4000)

In [0]:
from google.colab import files
files.download('test.mp4')